In [78]:
import simpm.des as des

env=des.Environment()


In [79]:
car1=des.Entity(env,"car1",True)
car2=des.Entity(env,"car2",True)
car3=des.Entity(env,"car3",True)
lot=des.Resource(env,"parking-lot",init=2,print_actions=True,log=True)

car1(1) is created, sim_time: 0
car2(2) is created, sim_time: 0
car3(3) is created, sim_time: 0


In [80]:
def car_process(carent,lot):
    while True:
        yield carent.get(lot,1)
        yield carent.do("park",5)
        yield carent.put(lot,1)
        yield carent.do("drive",3)



In [81]:
env.process(car_process(car1,lot))
env.process(car_process(car2,lot))
env.process(car_process(car3,lot))
env.run(until=100)

car1(1) requested 1 parking-lot(s)(1), sim_time: 0
car2(2) requested 1 parking-lot(s)(1), sim_time: 0
car3(3) requested 1 parking-lot(s)(1), sim_time: 0
car1(1) got 1 parking-lot(s)(1), sim_time: 0
car2(2) got 1 parking-lot(s)(1), sim_time: 0
car1(1) started park , sim_time: 0
car2(2) started park , sim_time: 0
car1(1) finished park , sim_time: 5
car2(2) finished park , sim_time: 5
car1(1) put back 1 parking-lot(s)(1), sim_time: 5
car2(2) put back 1 parking-lot(s)(1), sim_time: 5
car1(1) started drive , sim_time: 5
car3(3) got 1 parking-lot(s)(1), sim_time: 5
car2(2) started drive , sim_time: 5
car3(3) started park , sim_time: 5
car1(1) finished drive , sim_time: 8
car2(2) finished drive , sim_time: 8
car1(1) requested 1 parking-lot(s)(1), sim_time: 8
car2(2) requested 1 parking-lot(s)(1), sim_time: 8
car1(1) got 1 parking-lot(s)(1), sim_time: 8
car1(1) started park , sim_time: 8
car3(3) finished park , sim_time: 10
car3(3) put back 1 parking-lot(s)(1), sim_time: 10
car3(3) started dri

In [93]:
car3.waiting_time().sum()
env.now

5

In [83]:
lot.waiting_time().mean()


0.18421052631578946

Earth moving example 
* We have 3 trucks
* Cpacity of trucks is 10 m3
* Loading Time is 10 minutes
* Haul Time(Minutes) is 30 minutes
* Dump Time (Minutes): 5 minutes
* Return Time (Minutes):10 minutes 
How long does it take to dump 10000 m3 of dirt? 

In [103]:
import simpm.des as des
env=des.Environment()

t1=des.Entity(env,"truck",True)
t2=des.Entity(env,"truck",True)
t3=des.Entity(env,"truck",True)
t4=des.Entity(env,"truck",True)
t5=des.Entity(env,"truck",True)

dirt=des.Resource(env,"dirt",init=10000,capacity=10000)
loader=des.Resource(env,"loader",init=2)
dump_station=des.Resource(env,"dump_station",init=1)

def truck_process(truck_e,loader,dump_station,dirt):
    while True:
        yield truck_e.get(loader,1)
        yield truck_e.get(dirt,10)
        yield truck_e.do("loading",10)
        yield truck_e.put(loader,1)
        yield truck_e.do("hauling",30)
        yield truck_e.get(dump_station,1)
        yield truck_e.do("dumping",5)
        yield truck_e.put(dump_station,1)
        yield truck_e.do("returning",10)


env.process(truck_process(t1,loader,dump_station,dirt))
env.process(truck_process(t2,loader,dump_station,dirt))
env.process(truck_process(t3,loader,dump_station,dirt))
env.process(truck_process(t4,loader,dump_station,dirt))
env.process(truck_process(t5,loader,dump_station,dirt))

env.run()        


truck(1) is created, sim_time: 0
truck(2) is created, sim_time: 0
truck(3) is created, sim_time: 0
truck(4) is created, sim_time: 0
truck(5) is created, sim_time: 0
truck(1) requested 1 loader(s)(2), sim_time: 0
truck(2) requested 1 loader(s)(2), sim_time: 0
truck(3) requested 1 loader(s)(2), sim_time: 0
truck(4) requested 1 loader(s)(2), sim_time: 0
truck(5) requested 1 loader(s)(2), sim_time: 0
truck(1) got 1 loader(s)(2), sim_time: 0
truck(2) got 1 loader(s)(2), sim_time: 0
truck(1) requested 10 dirt(s)(1), sim_time: 0
truck(2) requested 10 dirt(s)(1), sim_time: 0
truck(1) got 10 dirt(s)(1), sim_time: 0
truck(2) got 10 dirt(s)(1), sim_time: 0
truck(1) started loading , sim_time: 0
truck(2) started loading , sim_time: 0
truck(1) finished loading , sim_time: 10
truck(2) finished loading , sim_time: 10
truck(1) put back 1 loader(s)(2), sim_time: 10
truck(2) put back 1 loader(s)(2), sim_time: 10
truck(1) started hauling , sim_time: 10
truck(3) got 1 loader(s)(2), sim_time: 10
truck(2) s

In [104]:
env.now

11020